In [1]:
!pip install gradio opencv-python-headless numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
tor

In [2]:
import cv2
import numpy as np
import gradio as gr

In [3]:
# Histogram Equalization
def histogram_equalization(img):
    # Convert the image to HSV (Hue, Saturation, Value)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Apply histogram equalization to the V (Value) channel (intensity)
    hsv_img[..., 2] = cv2.equalizeHist(hsv_img[..., 2])

    # Convert back to BGR color space
    eq_color_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)

    return eq_color_img

In [4]:
#Gamma Transformation
def gamma_correction(img, gamma):
    table = np.array([((i / 255.0) ** gamma) * 255 for i in range(256)]).astype("uint8")
    gamma_corrected = cv2.LUT(img, table)
    return gamma_corrected

In [5]:
def increase_contrast(img, factor=5):
    # Convert the image to float32 to avoid clipping
    img_float = img.astype(np.float32)

    # Increase the contrast by multiplying the pixel values by a factor
    img_float = img_float * factor

    # Clip the values to be within the valid range [0, 255]
    img_float = np.clip(img_float, 0, 255)

    # Convert back to uint8
    contrast_img = img_float.astype(np.uint8)

    return contrast_img

In [6]:
# log trasnformation on the image
def log_transform(image):
    """Apply log transformation while preserving color"""
    image = np.float32(image) + 1
    c = 255 / np.log(1 + np.max(image))
    log_img = c * np.log(image)
    log_img = np.clip(log_img, 0, 255)
    return np.uint8(log_img)

In [7]:
# low light enhanced image
def enhance_image(image, gamma=0.5):
    """Enhance the image using multiple techniques"""
    contrast_img = increase_contrast(image)
    # log_img = log_transform(contrast_img)
    gamma_img = gamma_correction(image, gamma)

    # Combine the best results using weighted fusion
    enhanced_img = cv2.addWeighted(gamma_img, 0.6, contrast_img, 0.4, 0)

    return enhanced_img

In [8]:
def process_image(input_img, gamma):
    enhanced_img = enhance_image(input_img, gamma)
    return enhanced_img

# Create Gradio UI
interface = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="numpy"),  # Input image
        gr.Slider(0.1, 3.0, value=0.5, label="Gamma Correction")  # Slider for gamma adjustment
    ],
    outputs=gr.Image(type="numpy"),  # Output image
    title="Low-Light Image Enhancement",
    description="Upload a low-light image to enhance brightness and contrast.",
)

# Launch the app
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9fab7610c93a5f029a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
